<a href="https://colab.research.google.com/github/OleksiiLatypov/DRU_Data_Science_Fundamentals/blob/main/Tensorflow/US_Gasoline_and_Diesel_Priecs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

- A1 = Weekly U.S. All Grades All Formulations Retail Gasoline Prices (Dollars
per Gallon)
- A2 = Weekly U.S. All Grades Conventional Retail Gasoline Prices (Dollars per Gallon)
- A3 = Weekly U.S. All Grades Reformulated Retail Gasoline Prices (Dollars per Gallon)
- R1 = Weekly U.S. Regular All Formulations Retail Gasoline Prices (Dollars per Gallon)
- R2 = Weekly U.S. Regular Conventional Retail Gasoline Prices (Dollars per Gallon)
- R3 = Weekly U.S. Regular Reformulated Retail Gasoline Prices (Dollars per Gallon)
- M1 = Weekly U.S. Midgrade All Formulations Retail Gasoline Prices (Dollars per Gallon)
- M2 = Weekly U.S. Midgrade Conventional Retail Gasoline Prices (Dollars per Gallon)
- M3 = Weekly U.S. Midgrade Reformulated Retail Gasoline Prices (Dollars per Gallon)
- P1 = Weekly U.S. Premium All Formulations Retail Gasoline Prices (Dollars per Gallon)
- P2 = Weekly U.S. Premium Conventional Retail Gasoline Prices (Dollars per Gallon)
- P3 = Weekly U.S. Premium Reformulated Retail Gasoline Prices (Dollars per Gallon)
- D1 = Weekly U.S. No 2 Diesel Retail Prices (Dollars per Gallon)





In [7]:
path = '/content/sample_data/PET_PRI_GND_DCUS_NUS_W.csv'
df = pd.read_csv(path, delimiter=',')
df.shape

(1361, 14)

In [9]:
df.head(5)

,Date,A1,A2,A3,R1,R2,R3,M1,M2,M3,P1,P2,P3,D1
0,01/02/1995,1.127,1.104,1.231,1.079,1.063,1.167,1.170,1.159,1.298,1.272,1.250,1.386,1.104
1,01/09/1995,1.134,1.111,1.232,1.086,1.070,1.169,1.177,1.164,1.300,1.279,1.256,1.387,1.102
2,01/16/1995,1.126,1.102,1.231,1.078,1.062,1.169,1.168,1.155,1.299,1.271,1.249,1.385,1.100
3,01/23/1995,1.132,1.110,1.226,1.083,1.068,1.165,1.177,1.165,1.296,1.277,1.256,1.378,1.095
4,01/30/1995,1.131,1.109,1.221,1.083,1.068,1.162,1.176,1.163,1.291,1.275,1.255,1.370,1.090


In [10]:
# Extracting month from date column.
df.Date = df.Date.apply(pd.to_datetime)
df['Day'] = df['Date'].apply(lambda x:x.day)
df['Month'] = df['Date'].apply(lambda x:x.month)
df['Year'] = df['Date'].apply(lambda x:x.year)
df.drop('Date',axis=1,inplace=True)

In [11]:
df.head()

,A1,A2,A3,R1,R2,R3,M1,M2,M3,P1,P2,P3,D1,Day,Month,Year
0,1.127,1.104,1.231,1.079,1.063,1.167,1.170,1.159,1.298,1.272,1.250,1.386,1.104,2,1,1995
1,1.134,1.111,1.232,1.086,1.070,1.169,1.177,1.164,1.300,1.279,1.256,1.387,1.102,9,1,1995
2,1.126,1.102,1.231,1.078,1.062,1.169,1.168,1.155,1.299,1.271,1.249,1.385,1.100,16,1,1995
3,1.132,1.110,1.226,1.083,1.068,1.165,1.177,1.165,1.296,1.277,1.256,1.378,1.095,23,1,1995
4,1.131,1.109,1.221,1.083,1.068,1.162,1.176,1.163,1.291,1.275,1.255,1.370,1.090,30,1,1995


In [15]:
dummies = pd.get_dummies(df[['Year','Day','Month']],drop_first=True)
df_dummies = pd.concat([df[['A1','A2','A3','R1','R2','R3','M1', 'M2','M3','P1','P2','P3']],dummies],axis=1)
target = df['D1']

In [16]:
df.head(5)

,A1,A2,A3,R1,R2,R3,M1,M2,M3,P1,P2,P3,D1,Day,Month,Year
0,1.127,1.104,1.231,1.079,1.063,1.167,1.170,1.159,1.298,1.272,1.250,1.386,1.104,2,1,1995
1,1.134,1.111,1.232,1.086,1.070,1.169,1.177,1.164,1.300,1.279,1.256,1.387,1.102,9,1,1995
2,1.126,1.102,1.231,1.078,1.062,1.169,1.168,1.155,1.299,1.271,1.249,1.385,1.100,16,1,1995
3,1.132,1.110,1.226,1.083,1.068,1.165,1.177,1.165,1.296,1.277,1.256,1.378,1.095,23,1,1995
4,1.131,1.109,1.221,1.083,1.068,1.162,1.176,1.163,1.291,1.275,1.255,1.370,1.090,30,1,1995


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1361 entries, 0 to 1360
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      1361 non-null   float64
 1   A2      1361 non-null   float64
 2   A3      1361 non-null   float64
 3   R1      1361 non-null   float64
 4   R2      1361 non-null   float64
 5   R3      1361 non-null   float64
 6   M1      1361 non-null   float64
 7   M2      1361 non-null   float64
 8   M3      1361 non-null   float64
 9   P1      1361 non-null   float64
 10  P2      1361 non-null   float64
 11  P3      1361 non-null   float64
 12  D1      1361 non-null   float64
 13  Day     1361 non-null   int64  
 14  Month   1361 non-null   int64  
 15  Year    1361 non-null   int64  
dtypes: float64(13), int64(3)
memory usage: 170.2 KB


In [40]:
# Splitting data into training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_dummies,target,test_size=0.30)
# Splitting train set into training and validation sets.
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.20)

In [84]:
from tensorflow.keras import regularizers

model = Sequential()

model.add(Dense(10, activation='relu', kernel_initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1),
                bias_initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1),
                kernel_regularizer=regularizers.l2(0.001)))  # L2 regularization with a regularization parameter of 0.01
model.add(Dense(500, activation='relu', kernel_initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1),
                bias_initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1),
                kernel_regularizer=regularizers.l2(0.001)))  # L2 regularization with a regularization parameter of 0.01

model.add(Dropout(0.2))

model.add(Dense(1))

model.compile(optimizer='Adam', loss='mean_absolute_error')
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)


In [85]:
model.fit(x=X_train.values,y=y_train.values,
          validation_data=(X_val.values,y_val.values),
          batch_size=100,epochs=500,callbacks=[early_stop])

Epoch 1/500
8/8 [==============================] - 1s 32ms/step - loss: 10.0714 - val_loss: 3.7782
Epoch 2/500
8/8 [==============================] - 0s 8ms/step - loss: 4.7080 - val_loss: 1.0175
Epoch 3/500
8/8 [==============================] - 0s 10ms/step - loss: 3.8175 - val_loss: 0.9209
Epoch 4/500
8/8 [==============================] - 0s 10ms/step - loss: 3.0398 - val_loss: 1.0143
Epoch 5/500
8/8 [==============================] - 0s 7ms/step - loss: 2.5068 - val_loss: 0.9412
Epoch 6/500
8/8 [==============================] - 0s 10ms/step - loss: 2.2006 - val_loss: 0.9644
Epoch 7/500
8/8 [==============================] - 0s 8ms/step - loss: 1.8603 - val_loss: 0.9194
Epoch 8/500
8/8 [==============================] - 0s 7ms/step - loss: 1.5565 - val_loss: 0.8972
Epoch 9/500
8/8 [==============================] - 0s 9ms/step - loss: 1.3559 - val_loss: 0.9152
Epoch 10/500
8/8 [==============================] - 0s 7ms/step - loss: 1.2030 - val_loss: 0.9094
Epoch 11/500
8/8 [======